In [1]:
######################################     Modules     #######################################
# MODULES
import sys
from time import process_time
import pickle
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

sys.path.append('../.')
import verbose_module
sys.path.append('../../../.')
import pIRPgym

path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPgym/'
experiments_path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/First Phase/'

Experiments = [1,2]
Replicas = [1,2]

alphas = [0.1,0.2,0.4,0.6,0.8]
time_limits = [1,30,60,300,1800,3600]

init_times = {1:0.1,30:1,60:3,300:5,1800:5,3600:10}

Policies = ['NN','RCL']+[f'CG_{time_limit}' for time_limit in time_limits]+\
           [f'CG_{time_limit}_{alpha}' for time_limit in time_limits for alpha in alphas]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


In [2]:
Experiments_results = {('Instance','M/K'):[5,10]}
Experiments_results.update({(policy,item):[] for policy in Policies for item in ['eff','gaps']})

for experiment in Experiments:

    reliability = {policy:[] for policy in Policies}
    gaps = {policy:[] for policy in Policies}

    for replica in Replicas:
        with open(experiments_path+f'Experiment {experiment}/Replica {replica}/instance_information.pkl', 'rb') as file:
            inst_info = pickle.load(file)

        results = dict()
        for policy in Policies:
            with open(experiments_path+f'Experiment {experiment}/Replica {replica}/{policy}.pkl','rb') as file:
                results[policy] = pickle.load(file)

        for t in range(inst_info['inst_gen'].T):
            best_performance = min([results['NN'][t][1]]+\
                                   [results['RCL'][t][0]]+\
                                   [results[policy][t][1] for policy in Policies[2:]])

            for policy in Policies:
                loc = 1 if policy!='RCL' else 0
                
                if results[policy][t][loc] - best_performance < 0.01:
                    reliability[policy].append(1)
                else:
                    reliability[policy].append(0)
                    gaps[policy].append((results[policy][t][loc]-best_performance)/best_performance)

    # Storing the results
    for policy in Policies:
        Experiments_results[(policy,'eff')].append(round(sum(reliability[policy])/len(reliability[policy]),4))
        if len(gaps[policy]) > 0:
            Experiments_results[(policy,'gaps')].append(round(sum(gaps[policy])/len(gaps[policy]),4))
        else:
            Experiments_results[(policy,'gaps')].append(0)

df = pd.DataFrame(Experiments_results)
df.index = Experiments
df = df.style.format(dict.fromkeys(list(df.columns)[1:],"{:.2%}"))
display(df)

In [42]:
sizes = {1:5,2:10}
E_x_values = [f'Exp{exp} (M/K={size})' for exp,size in sizes.items()]
E_results = dict()

runtime = 1800
policies = ['NN','RCL',f'CG_{runtime}']+[f'CG_{runtime}_{alpha}' for alpha in alphas]

for policy in policies:
    E_results[policy] ={'eff':Experiments_results[str(policy),'eff'],'gaps':Experiments_results[str(policy),'gaps']} 

def plot_thing(indicator):
    pIRPgym.Visualizations.RoutingV.plot_indicator_evolution(E_results,indicator,x_axis='Experiments',x_values=E_x_values)

interact(plot_thing,indicator=widgets.Dropdown(options=['eff','gaps'],value='eff',description='Indicator:'))

interactive(children=(Dropdown(description='Indicator:', options=('eff', 'gaps'), value='eff'), Output()), _do…

<function __main__.plot_thing(indicator)>

In [61]:
with open(experiments_path+f'Experiment 3/Replica 1/CG_300_0.6.pkl','rb') as file:
        results = pickle.load(file)

In [62]:
results[1][4]

(23, 8)